### Challenge 10: Implement CBC mode

<div class="alert alert-block alert-info">
    
CBC mode is a block cipher mode that allows us to encrypt irregularly-sized messages, despite the fact that a block cipher natively only transforms individual blocks.

In CBC mode, each ciphertext block is added to the next plaintext block before the next call to the cipher core.

The first plaintext block, which has no associated previous ciphertext block, is added to a "fake 0th ciphertext block" called the initialization vector, or IV.

Implement CBC mode by hand by taking the ECB function you wrote earlier, making it encrypt instead of decrypt (verify this by decrypting whatever you encrypt to test), and using your XOR function from the previous exercise to combine them.

The file here is intelligible (somewhat) when CBC decrypted against "YELLOW SUBMARINE" with an IV of all ASCII 0 (\x00\x00\x00 &c)

<div class="alert alert-block alert-warning">
    
### Don't cheat.

Do not use OpenSSL's CBC code to do CBC mode, even to verify your results. What's the point of even doing this stuff if you aren't going to learn from it?
</div>
    
</div>

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

In [1]:
import cryptopals as cp
import base64

In [2]:
# Read the challenge file in.
f = open('./challenge-data/10.txt', 'r')
encrypted_data = base64.b64decode(f.read())
f.close()

Let's implement AES...ECB and CBC mode for now.  For ECB, we'll use Python's implementation from the crypto lib.  For the other modes, we'll wrap ECB mode and implement them ourselves.

In [3]:
from Crypto.Cipher import AES

# Set the (known) AES key
key = b"YELLOW SUBMARINE"
# Set the (known) AES CBC IV
IV = [0]*8

In [4]:
def AES_encrypt(plaintext, key, mode='ECB', IV=[0]*16):
    """Implement Advanced Encryption Standard (AES) Encryption"""
    plaintext = PKCS7_pad(plaintext, 16)

    blockSize = 16
    ciphertext = b''

    aes = AES.new(key, AES.MODE_ECB)

    if mode == 'ECB':

        ciphertext = aes.encrypt(plaintext)

    elif mode == 'CBC':

        PT_Blocks = [plaintext[ii:ii+blockSize] for ii in
                     range(0, len(plaintext), blockSize)]

        for block in PT_Blocks:

            AES_input = cp.bitwise_xor(block, IV)
            IV = aes.encrypt(AES_input)
            ciphertext += IV

    else:

        assert(f'Mode {mode} is not supported yet!')

    return(ciphertext)


def AES_decrypt(ciphertext, key, mode='ECB', IV=[0]*16):
    """Implement Advanced Encryption Standard (AES) Decryption"""
    blockSize = 16
    plaintext = b''

    aes = AES.new(key, AES.MODE_ECB)

    if mode == 'ECB':

        return(aes.decrypt(ciphertext))

    elif mode == 'CBC':

        CT_Blocks = [ciphertext[ii:ii+blockSize] for ii in
                     range(0, len(ciphertext), blockSize)]

        for block in CT_Blocks:

            AES_output = aes.decrypt(block)
            plaintext += cp.bitwise_xor(AES_output, IV)
            IV = block
            
    else:

        assert(f'Mode {mode} is not supported yet!')

    return(plaintext)

In [6]:
plaintext = AES_decrypt(encrypted_data, key, 'CBC')
print(plaintext.decode())

Test out encryption and decryption with new random key and IV.

In [14]:
from Crypto import Random
import cryptopals as cp

key = Random.get_random_bytes(32)
rand_IV = Random.get_random_bytes(16)
ciphertext = cp.AESEncrypt(plaintext, key, 'CBC', rand_IV)
plaintext_2 = cp.AESDecrypt(ciphertext, key, 'CBC', rand_IV)

print(cp.strip_PKCS7_pad(plaintext_2).decode())

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)